In [1]:
import os
import torch
import numpy as np
import cv2
import json
from argparse import Namespace
import tools._init_paths
from core.inference import get_final_preds
from config import cfg, update_config
from models.adaptive_pose_hrnet import get_adaptive_pose_net
from utils.transforms import get_affine_transform
from utils.vis import save_debug_images
from torchvision.transforms import Normalize, ToTensor
import pickle

np.set_printoptions(precision=2, suppress=True)

/home/dresden/miniconda3/envs/fidip/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dresden/miniconda3/envs/fidip/lib/python3.12/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971214/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [26]:
def GetSyripKP(path: str,id: int):
    with open(path,'r') as f:
        ann = json.load(f)
    if(not isinstance(ann,list)):
        ann = ann['annotations']
    skp = next(obj for obj in ann if obj['image_id'] == id)['keypoints']
    return np.array(skp).reshape((-1,3))
def WriteKP(path,id,joints):
    print(f'{id}: {joints.shape}')
    joints[:,2] = 2.0
    with open(f'{path}/{id}_keypoints.json','w') as f:
        json.dump({'annotations': joints.flatten().tolist()},f)

In [19]:
syrip_ref_path = 'data/syrip/annotations/person_keypoints_validate_infant.json'
syrip_output_path = 'output/syrip/adaptive_pose_hrnet/w48_384x288_adam_lr1e-3_infant/results/keypoints_validate_infant_results_0.json'
custom_output_path = 'output/custom/adaptive_pose_hrnet/test-infant/results/keypoints_predict_results_0.json'

In [27]:
ids = ['1','2','4','5','6','7','10']
joints = {}
syn_output_path = 'syn_generation/data/keypoints'
for id in ids:
    joints[id] = GetSyripKP(custom_output_path,id)
for id,j in joints.items():
    WriteKP(syn_output_path,id,j)

1: (19, 3)
2: (19, 3)
4: (19, 3)
5: (19, 3)
6: (19, 3)
7: (19, 3)
10: (19, 3)


In [5]:
# sref_kp = GetSyripKP(syrip_ref_path,10)
# sout_kp = GetSyripKP(syrip_output_path,10)
# custom_kp = GetSyripKP(custom_output_path,'10')

In [ ]:
# print(sref_kp.shape)
# sref_kp

In [ ]:
# print(sout_kp.shape)
# sout_kp

In [ ]:
# print(custom_kp.shape)
# custom_kp

In [9]:
# joints = custom_kp
# coco_joints = {
#     0: "nose",
#     1: "left_eye",
#     2: "right_eye",
#     3: "left_ear",
#     4: "right_ear",
#     5: "left_shoulder",
#     6: "right_shoulder",
#     7: "left_elbow",
#     8: "right_elbow",
#     9: "left_wrist",
#     10: "right_wrist",
#     11: "left_hip",
#     12: "right_hip",
#     13: "left_knee",
#     14: "right_knee",
#     15: "left_ankle",
#     16: "right_ankle",
#     17: "neck",  # computed value, not in original coco format
#     18: "mid_hip" # computed value, not in original coco format
# }
# op_joints ={
#     0:  "nose",
#     1:  "neck",
#     2:  "right_shoulder",
#     3:  "right_elbow",
#     4:  "right_wrist",
#     5:  "left_shoulder",
#     6:  "left_elbow",
#     7:  "left_wrist",
#     8:  "mid_hip",
#     9:  "right_hip",
#     10: "right_knee",
#     11: "right_ankle",
#     12: "left_hip",
#     13: "left_knee",
#     14: "left_ankle",
#     15: "right_eye",
#     16: "left_eye",
#     17: "right_ear",
#     18: "left_ear",
#     # 19: "background" # not used
# }

# # compute neck and mid_hip
# neck = (joints[5] + joints[6]) / 2.0
# mid_hip = (joints[11] + joints[12]) / 2.0
# joints = np.vstack((joints,neck,mid_hip))

# # remap from coco to openpose
# coco_to_op_map = []
# for op_idx, op_joint in op_joints.items():
#     # Find the corresponding COCO index
#     coco_idx = next((coco_idx for coco_idx, coco_joint in coco_joints.items() if coco_joint == op_joint), None)
#     coco_to_op_map.append(coco_idx)
# tmp = np.zeros((19,3))
# for op_idx, coco_idx in enumerate(coco_to_op_map):
#         if coco_idx is not None:
#             tmp[op_idx,:] = joints[coco_idx,:]

# joints = tmp

In [10]:
# with open('syn_generation/data/keypoints/custom_keypoints.json','w') as f:
#     json.dump({'annotations': joints.flatten().tolist()},f)

In [2]:
with open('syn_generation/output/results/custom/000.pkl','rb') as f:
    data = pickle.load(f,encoding='latin1')

In [3]:
data

{'camera_rotation': array([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]], dtype=float32),
 'camera_translation': array([[-0.05, -0.08,  0.85]], dtype=float32),
 'betas': array([[-1.  , -0.34,  0.16, -0.02,  0.  , -0.04,  0.03, -0.09, -0.06,
          0.01, -0.01,  0.03, -0.02, -0.  , -0.01, -0.01, -0.01,  0.  ,
          0.  ,  0.  ]], dtype=float32),
 'global_orient': array([[-1.01,  1.04, -2.35]], dtype=float32),
 'body_pose': array([[-1.03,  0.45,  1.21, -1.58, -0.15, -0.31,  0.7 , -0.18, -0.28,
          1.21,  0.35, -0.11, -0.06, -0.26,  0.17, -0.11, -0.1 , -0.1 ,
          0.2 ,  0.13, -0.3 , -0.04, -0.2 ,  0.2 ,  0.08, -0.11, -0.06,
         -0.24, -0.01, -0.07, -0.01,  0.08, -0.63, -0.4 , -0.8 , -0.13,
          0.12,  0.23, -0.22, -0.01,  0.1 ,  0.29, -0.26, -0.84,  0.33,
          0.32,  0.28, -0.62, -0.09,  0.39,  0.61,  0.29, -0.81, -0.27,
          0.28,  0.32,  0.33,  0.06,  0.07, -0.17, -0.01, -0.07, -0.09,
         -0.13,  0.05,  0.17,  0.11, -0.07, -0.1